# Problem Definition for RFM Analysis
📌 The UK-based retail company wants to divide its customers into segments and determine marketing strategies according to these segments. He thinks that conducting marketing studies specifically for customer segments that exhibit common behaviors will increase revenue. RFM analysis will be used to segment.

## Import necessary libraries

In [41]:
!pip install lifetimes
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
pd.set_option("display.max_columns",None)
pd.set_option("display.width",500)
pd.set_option("display.float_format", lambda x: '%.4f' % x)
online_retail = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/online_retail_II.xlsx",sheet_name="Year 2010-2011")
df = online_retail.copy()
df.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom


## General Information About Dataset

In [42]:
def check_df(dataframe,head=10):
  print("################################ Head ################################\n")
  print(dataframe.head(head))
  print("################################ Tail ################################\n")
  print(dataframe.tail(head))
  print("################################ Shape ################################\n")
  print(dataframe.shape)
  print("################################ Types ################################\n")
  print(dataframe.dtypes)
  print("################################ NA ################################\n")
  print(dataframe.isnull().sum())
  print("################################ Qurtiles ################################\n")
  print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

################################ Head ################################

  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00 2.5500   17850.0000  United Kingdom
1  536365     71053                  WHITE METAL LANTERN         6 2010-12-01 08:26:00 3.3900   17850.0000  United Kingdom
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00 2.7500   17850.0000  United Kingdom
3  536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6 2010-12-01 08:26:00 3.3900   17850.0000  United Kingdom
4  536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6 2010-12-01 08:26:00 3.3900   17850.0000  United Kingdom
5  536365     22752         SET 7 BABUSHKA NESTING BOXES         2 2010-12-01 08:26:00 7.6500   17850.0000  United Kingdom
6  536365     21730    GLASS STAR FROSTED T-LIGHT HOLDER         6 

## Create RFM

In [43]:
def create_rfm(dataframe,csv=False):
  # Data Preprocessing
  dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
  dataframe.dropna(inplace=True)
  dataframe = dataframe[~dataframe["Invoice"].str.contains("C",na=False)]
  dataframe = dataframe[(dataframe["Quantity"] > 0)]

  # Calculating RFM Metrics
  today_date = dt.datetime(2011,12,11)
  rfm = dataframe.groupby(["Customer ID"]).agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                                'Invoice': lambda Invoice: Invoice.nunique(),
                                                'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
  rfm.columns = ["recency","frequency","monetary"]
  rfm =rfm[rfm["monetary"] > 0]

  # Calculating RFM Scores
  rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5,4,3,2,1])
  rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
  rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1,2,3,4,5])
  rfm["RF_SCORE"] = rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str)

  # Set RFM Names
  seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_to_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loaylists',
    r'5[4-5]': 'champions'
  }
  rfm["segment"] = rfm["RF_SCORE"].replace(seg_map, regex=True)
  rfm.index = rfm.index.astype(int)

  # Save Results to CSV File
  if csv:
    rfm.to_csv("rfm.csv")
  
  return rfm


In [44]:
rfm = create_rfm(df)

**Q**: Select the customer IDs of the "Loyal Customers" class and get the excel output.

In [45]:
loyal_customer_ids = rfm[rfm["segment"].isin(["loyal_customers"])].index
loyal_customer_ids = pd.DataFrame(loyal_customer_ids)
loyal_customer_ids.to_csv("loyal_customer.csv")

# Problem Definition for CLTV Prediction
📌 The UK-based retail company wants to set a roadmap for its sales and marketing activities. In order for the company to make a medium-long-term plan, it is necessary to estimate the potential value that existing customers will provide to the company in the future.

## Find Outlier and then replace with thresholds

In [46]:
def outlier_thresholds(dataframe,variable):
  quartile1 = dataframe[variable].quantile(0.01)
  quartile3 = dataframe[variable].quantile(0.99)
  interquartile_range = quartile3 - quartile1
  up_limit = quartile3 + 1.5 * interquartile_range
  low_limit = quartile1 - 1.5 * interquartile_range
  return low_limit, up_limit

def replace_with_thresholds(dataframe,variable):
  low_limit, up_limit = outlier_thresholds(dataframe,variable)
  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
  dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

## Create CLTV Prediction

In [47]:
def create_cltv_prediction(dataframe,month=3,csv=False):
  # Data Preprocessing
  dataframe.dropna(inplace=True)
  dataframe = dataframe[~dataframe["Invoice"].str.contains("C",na=False)]
  dataframe = dataframe[dataframe["Quantity"] > 0]
  dataframe = dataframe[dataframe["Price"] > 0]
  replace_with_thresholds(dataframe,"Quantity")
  replace_with_thresholds(dataframe,"Price")
  dataframe["TotalPrice"] = dataframe["Price"] * dataframe["Quantity"]
  today_date = dt.datetime(2011,12,11)

  # Preparation of lifetime data structure
  cltv_df = dataframe.groupby(["Customer ID"]).agg({'InvoiceDate': [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,
                                                                    lambda InvoiceDate: (today_date - InvoiceDate.min()).days],
                                                    'Invoice': lambda Invoice: Invoice.nunique(),
                                                    'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
  cltv_df.columns = cltv_df.columns.droplevel(0)
  cltv_df.columns = ["recency","T","frequency","monetary"]
  cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]
  cltv_df = cltv_df[(cltv_df["frequency"] > 1)]
  cltv_df["recency"] = cltv_df["recency"] / 7
  cltv_df["T"] = cltv_df["T"] / 7

  # Set the BG-NBD Model
  bgf = BetaGeoFitter(penalizer_coef=0.001)
  bgf.fit(cltv_df["frequency"],cltv_df["recency"],cltv_df["T"])

  cltv_df["expected_purc_1_week"] = bgf.predict(1,cltv_df["frequency"],cltv_df["recency"],cltv_df["T"])
  cltv_df["expected_purc_1_month"] = bgf.predict(4*1,cltv_df["frequency"],cltv_df["recency"],cltv_df["T"])
  cltv_df["expected_purc_6_month"] = bgf.predict(4*6,cltv_df["frequency"],cltv_df["recency"],cltv_df["T"])
  cltv_df["expected_purc_12_month"] = bgf.predict(4*12,cltv_df["frequency"],cltv_df["recency"],cltv_df["T"])

  # Set the Gamma-Gamma Model
  ggf = GammaGammaFitter(penalizer_coef=0.01)
  ggf.fit(cltv_df["frequency"],cltv_df["monetary"])

  cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df["frequency"],cltv_df["monetary"])

  # Calculating cltv with BG-NBD Model and Gamma-Gamma Model
  cltv = ggf.customer_lifetime_value(bgf,
                                     cltv_df["frequency"],
                                     cltv_df["recency"],
                                     cltv_df["T"],
                                     cltv_df["monetary"],
                                     time=month, # month
                                     freq="W",
                                     discount_rate=0.01)
  cltv = cltv.reset_index()
  cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")

  # Creation of Segment
  cltv_final["segment"] = pd.qcut(cltv_final["clv"], 4, labels=["D","C","B","A"])

  # Export All Results to CSV File
  if csv:
    cltv_final.to_csv("cltv_prediction.csv")
  
  return cltv_final

In [49]:
create_cltv_prediction(df,month=6,csv=True)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_6_month,expected_purc_12_month,expected_average_profit,clv,segment
0,12347.0000,52.1429,52.5714,7,615.7143,0.1413,0.5635,3.3239,6.5358,631.9123,2200.7340,A
1,12348.0000,40.2857,51.2857,4,442.6950,0.0920,0.3668,2.1618,4.2483,463.7460,1050.3748,B
2,12352.0000,37.1429,42.4286,8,219.5425,0.1824,0.7271,4.2775,8.3928,224.8868,1007.7137,B
3,12356.0000,43.1429,46.5714,3,937.1433,0.0862,0.3435,2.0219,3.9684,995.9989,2109.6204,A
4,12358.0000,21.2857,21.5714,2,575.2100,0.1223,0.4862,2.8279,5.5024,631.9022,1870.5449,A
...,...,...,...,...,...,...,...,...,...,...,...,...
2840,18272.0000,34.8571,35.2857,6,513.0967,0.1721,0.6856,4.0220,7.8746,529.0185,2228.5118,A
2841,18273.0000,36.4286,36.8571,3,68.0000,0.1043,0.4157,2.4390,4.7751,73.4942,187.7430,D
2842,18282.0000,16.8571,18.1429,2,89.0250,0.1357,0.5392,3.1274,6.0743,99.5249,325.7606,D
2843,18283.0000,47.5714,48.2857,16,130.9300,0.3017,1.2034,7.0936,13.9412,132.6012,985.5124,C


In [50]:
cltv_final = create_cltv_prediction(df,month=6,csv=True)
cltv_final.groupby(["segment"]).agg({'clv': ['mean','min','max']})

clv                      
             mean       min         max
segment                                
D        278.5765    0.0000    515.6869
C        740.0285  515.8744    987.1265
B       1335.9295  987.5873   1792.4893
A       4317.3349 1792.8921 108728.3152

**Q**: Analyze the 10 highest customers at 1 month CLTV and the 10 highest at 12 months.

In [56]:
cltv_final["expected_purc_1_month"].sort_values(ascending=False).head(10)

215    12.9633
1257   12.4722
2652    7.7398
373     6.1330
1101    5.8399
1441    5.7191
319     5.4131
1122    4.8119
511     3.9341
2761    3.8636
Name: expected_purc_1_month, dtype: float64

In [57]:
cltv_final["expected_purc_12_month"].sort_values(ascending=False).head(10)

215    150.6340
1257   144.9265
2652    89.9322
373     71.2436
1101    67.8534
1441    66.4497
319     62.8905
1122    55.8439
511     45.7070
2761    44.8717
Name: expected_purc_12_month, dtype: float64